In [7]:
# STEP 1 — Install Kaggle CLI and setup credentials
!pip install kaggle --quiet

import os
from pathlib import Path

# Ensure kaggle credentials exist
kaggle_path = Path.home() / ".kaggle"
kaggle_json = kaggle_path / "kaggle.json"

if not kaggle_json.exists():
    raise FileNotFoundError(
        f"❌ kaggle.json not found at {kaggle_json}. Please create it via Kaggle account -> Settings -> Create API Token."
    )

# Set permissions (especially important on Windows)
os.chmod(kaggle_json, 0o600)

print("✅ Kaggle API key found and ready.")


✅ Kaggle API key found and ready.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# STEP 2 — Download the Kaggle dataset
!kaggle datasets download -d anirudhchauhan/retail-store-inventory-forecasting-dataset -p data

# Unzip the downloaded file
import zipfile

zip_path = Path("data/retail-store-inventory-forecasting-dataset.zip")
extract_to = Path("data")
if zip_path.exists():
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Dataset extracted to 'data/' folder.")
else:
    print("❌ Zip file not found. Kaggle download may have failed.")


Dataset URL: https://www.kaggle.com/datasets/anirudhchauhan/retail-store-inventory-forecasting-dataset
License(s): CC0-1.0

✅ Dataset extracted to 'data/' folder.



  0%|          | 0.00/1.51M [00:00<?, ?B/s]
100%|##########| 1.51M/1.51M [00:00<00:00, 430MB/s]


In [10]:
import os

# List all files in the "data" folder after extraction
for root, dirs, files in os.walk("data"):
    for name in files:
        print(os.path.join(root, name))



data\retail-store-inventory-forecasting-dataset.zip
data\retail_store_inventory.csv


In [12]:
import os, shutil
from pathlib import Path

data_path = Path("data")
dst = data_path / "retail_inventory.csv"

# Auto-detect any CSV file except the processed or zip
csv_files = [f for f in data_path.glob("*.csv") if f.name != "retail_inventory.csv"]

if csv_files:
    src = csv_files[0]  # take the first CSV file found
    shutil.move(str(src), str(dst))
    print(f"✅ Renamed '{src.name}' → 'retail_inventory.csv'")
else:
    print("⚠️ No CSV file found in 'data/' after extraction.")


✅ Renamed 'retail_store_inventory.csv' → 'retail_inventory.csv'


In [13]:
import pandas as pd

df = pd.read_csv("data/retail_inventory.csv")
print("✅ Loaded successfully:", df.shape)
display(df.head())


✅ Loaded successfully: (73100, 15)


,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn
2,2022-01-01,S001,P0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer
3,2022-01-01,S001,P0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Autumn
4,2022-01-01,S001,P0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Summer


In [14]:
# DEADSTOCK AI REDISTRIBUTION — FULL PIPELINE (AUTO-DOWNLOAD + PROCESS)
# ====================================================================
# - Installs/authenticates Kaggle CLI
# - Downloads and extracts the dataset
# - Auto-detects CSV name and renames to data/retail_inventory.csv
# - Flags deadstock (high inventory + low sales)
# - Builds TF-IDF embeddings (for quick sanity checks)
# - Saves backend/data/processed_inventory.csv (for FastAPI backend)

# -----------------------------
# 0) Imports
# -----------------------------
import os
from pathlib import Path
import zipfile
import shutil
import pandas as pd
import numpy as np

# Optional (for a quick sanity check / preview embeddings)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# -----------------------------
# 1) Kaggle setup (run once)
# -----------------------------
# If Kaggle CLI not installed, install it.
try:
    import kaggle  # type: ignore
except ImportError:
    !pip install kaggle --quiet

# Check for Kaggle API key
kaggle_dir = Path.home() / ".kaggle"
kaggle_json = kaggle_dir / "kaggle.json"
if not kaggle_json.exists():
    raise FileNotFoundError(
        f"❌ kaggle.json not found at {kaggle_json}\n"
        "Create it from Kaggle: Account → Settings → Create New API Token,\n"
        f"then place the file at: {kaggle_json}"
    )

# Set restrictive permissions (works on Windows too; harmless if already set)
try:
    os.chmod(kaggle_json, 0o600)
except Exception:
    pass

print("✅ Kaggle API key detected.")

# -----------------------------
# 2) Download + extract dataset
# -----------------------------
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

zip_path = data_dir / "retail-store-inventory-forecasting-dataset.zip"
csv_target = data_dir / "retail_inventory.csv"

# Only download if we don't already have the final CSV
if not csv_target.exists():
    print("⬇️  Downloading dataset from Kaggle …")
    # Download ZIP to data/
    !kaggle datasets download -d anirudhchauhan/retail-store-inventory-forecasting-dataset -p data -q

    if not zip_path.exists():
        raise FileNotFoundError("❌ Download failed: zip not found at " + str(zip_path))

    # Extract all
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(data_dir)
    print("✅ Extracted to:", data_dir)

    # Auto-detect the extracted CSV file (whatever the name is), rename to retail_inventory.csv
    csv_candidates = [p for p in data_dir.glob("*.csv")]
    if not csv_candidates:
        raise FileNotFoundError("❌ No CSV found after extraction in 'data/'.")
    # Prefer the largest CSV if multiple exist
    src_csv = max(csv_candidates, key=lambda p: p.stat().st_size)
    shutil.move(str(src_csv), str(csv_target))
    print(f"✅ Renamed '{src_csv.name}' → '{csv_target.name}'")
else:
    print("ℹ️  Skipping download: 'data/retail_inventory.csv' already exists.")

# -----------------------------
# 3) Load dataset
# -----------------------------
df = pd.read_csv(csv_target)
print("✅ Loaded dataset:", csv_target, "shape:", df.shape)
display(df.head(3))
print("Columns:", df.columns.tolist())

# -----------------------------
# 4) Clean + deadstock flag
# -----------------------------
df.columns = [c.strip() for c in df.columns]  # normalize whitespace

# Ensure numeric conversions
if 'Inventory Level' not in df.columns or 'Units Sold' not in df.columns:
    raise KeyError(
        "Expected columns 'Inventory Level' and 'Units Sold' not found. "
        f"Columns present: {df.columns.tolist()}"
    )

df['Inventory Level'] = pd.to_numeric(df['Inventory Level'], errors='coerce')
df['Units Sold']      = pd.to_numeric(df['Units Sold'], errors='coerce')

mean_inventory = df['Inventory Level'].mean()
mean_sales     = df['Units Sold'].mean()

# Heuristic: high inv (>1.5× mean) AND low sales (<0.5× mean)
df['deadstock_flag'] = (
    (df['Inventory Level'] > mean_inventory * 1.5) &
    (df['Units Sold']      < mean_sales * 0.5)
)

print("\n📦 Deadstock counts:")
print(df['deadstock_flag'].value_counts(dropna=False))

# -----------------------------
# 5) Build text_feature for embeddings
# -----------------------------
text_cols = ['Category', 'Region', 'Weather Condition', 'Holiday/Promotion', 'Competitor Pricing']
for c in text_cols:
    if c not in df.columns:
        df[c] = ''  # create empty if missing
    # force to string, fill NaN
    df[c] = df[c].astype(str).fillna('')

df['text_feature'] = (
    df['Category'] + ' ' +
    df['Region'] + ' ' +
    df['Weather Condition'] + ' ' +
    df['Holiday/Promotion'] + ' ' +
    df['Competitor Pricing']
).str.strip()

print("\n✅ text_feature created.")
display(df[['Category','Region','text_feature']].head(3))

# -----------------------------
# 6) (Optional) quick sanity: TF-IDF + nearest neighbours
# -----------------------------
vec = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
emb = vec.fit_transform(df['text_feature'])
print("Embeddings shape:", emb.shape)

try:
    nn = NearestNeighbors(n_neighbors=5, metric='cosine').fit(emb)
    print("✅ NearestNeighbors index ready.")
except Exception as e:
    print("⚠️ Skipping NN sanity check:", e)

# -----------------------------
# 7) Normalize columns for backend + save processed CSV
# -----------------------------
# Rename to snake_case/lowercase for backend
rename_map = {
    'Product ID': 'product_id',
    'Category': 'category',
    'Region': 'region',
    'Inventory Level': 'inventory_level',
    'Units Sold': 'units_sold',
    # keep 'deadstock_flag' and 'text_feature' names as-is
}
for old, new in rename_map.items():
    if old in df.columns:
        df.rename(columns={old: new}, inplace=True)

backend_data_dir = Path("backend/data")
backend_data_dir.mkdir(parents=True, exist_ok=True)
out_csv = backend_data_dir / "processed_inventory.csv"

df.to_csv(out_csv, index=False)
print(f"\n✅ Processed dataset saved → {out_csv}")

# -----------------------------
# 8) Final validation (read back)
# -----------------------------
check = pd.read_csv(out_csv)
print("✅ Read-back OK. Columns:", check.columns.tolist())
display(check.head(3))


✅ Kaggle API key detected.
ℹ️  Skipping download: 'data/retail_inventory.csv' already exists.
✅ Loaded dataset: data\retail_inventory.csv shape: (73100, 15)


,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn
2,2022-01-01,S001,P0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer


Columns: ['Date', 'Store ID', 'Product ID', 'Category', 'Region', 'Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount', 'Weather Condition', 'Holiday/Promotion', 'Competitor Pricing', 'Seasonality']

📦 Deadstock counts:
deadstock_flag
False    70836
True      2264
Name: count, dtype: int64

✅ text_feature created.


,Category,Region,text_feature
0,Groceries,North,Groceries North Rainy 0 29.69
1,Toys,South,Toys South Sunny 0 66.16
2,Toys,West,Toys West Sunny 1 31.32


Embeddings shape: (73100, 5000)
✅ NearestNeighbors index ready.

✅ Processed dataset saved → backend\data\processed_inventory.csv
✅ Read-back OK. Columns: ['Date', 'Store ID', 'product_id', 'category', 'region', 'inventory_level', 'units_sold', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount', 'Weather Condition', 'Holiday/Promotion', 'Competitor Pricing', 'Seasonality', 'deadstock_flag', 'text_feature']


,Date,Store ID,product_id,category,region,inventory_level,units_sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,deadstock_flag,text_feature
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,False,Groceries North Rainy 0 29.69
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn,False,Toys South Sunny 0 66.16
2,2022-01-01,S001,P0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer,False,Toys West Sunny 1 31.32
